In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# global imports

import sys
import time
import pickle
import numpy as np
import pandas as pd
import scipy.stats as stats
from os import path
import glob
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import TwoSlopeNorm
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# local imports
import utils

# general settings  
plt.style.use('seaborn-whitegrid')
plt.rc('font', size=14)
plt.rc('legend', fontsize=14)
plt.rc('lines', linewidth=2)
plt.rc('axes', linewidth=2)
plt.rc('axes', edgecolor='k')
plt.rc('xtick.major', width=2)
plt.rc('xtick.major', size=6)
plt.rc('ytick.major', width=2)
plt.rc('ytick.major', size=6)
plt.rc('pdf', fonttype=42)
plt.rc('ps', fonttype=42)

In [ ]:
%%time
# subset of attributes
attributes = utils.get_attributes('subset1')
print('Attributes', attributes)
states = utils.states
nstates = len(states)
print('No States', nstates)
state2pos = {s:i for i, s in enumerate(states)}

# load distances (see 01_CalculateDistances.ipynb)
dists = pickle.load( open("results/distances.pkl", "rb" ) )
# restrict to subset and states
dists = { k:{att:d for att, d in v.items() if att in attributes} for k, v in dists.items() if k[0] in states and k[1] in states and k[0] != k[1]}

In [ ]:
# load results
def load_res(files):
    results = []
    for filename in glob.glob(files):
        print(filename)
        result = pickle.load( open( filename, "rb" ) )
        results.extend(result)
    return results

acc_res = load_res("results/exp/results_*_subset1_8.pkl")
#acc_res2 = load_res("results/mp_05_td_2", "acc")
#acc_res1 = load_res("results/mp_05_td_1", "acc")
#dp_res = load_res("results/mp_05_td_8", "dp")
#dp_res2 = load_res("results/mp_05_td_2", "dp")
#dp_res1 = load_res("results/mp_05_td_1", "dp")
#eop_res = load_res("results/mp_05_td_8", "eop")
#eop_res2 = load_res("results/mp_05_td_2", "eop")
#eop_res1 = load_res("results/mp_05_td_1", "eop")

In [ ]:
# look at an example
[{k:v for k, v in r.items() if k != 'clf'} for r in acc_res if r['source']=='AL' and r['target']=='OR']

In [ ]:
# find specific results
def cond(l, source=None, target=None, da=False):
    return [r for r in l if (source is None or (r['source']==source)) and\
            (target is None or (r['target']==target)) and (r['da']==da) ]

# build metric matrix
def metric_m(results, met, kind='alpha_rel', diag=False):
    res = np.zeros((nstates, nstates))
    for i, source in enumerate(states):
        for j, target in enumerate(states):
            if i==j and not diag:
                res[i, j] = np.NaN
                continue
            target_target = cond(results, source=target, target=target, da=False)
            source_target = cond(results, source=source, target=target, da=False)
            source_target_da = cond(results, source=source, target=target, da=True)
            if len(target_target)==1 and len(source_target)==1 and len(source_target_da)==1:
                target_target_v = utils.get_metric(target_target[0], met)
                source_target_v = utils.get_metric(source_target[0], met)
                source_target_da_v = utils.get_metric(source_target_da[0], met)                    
                mi = min(target_target_v, source_target_v)
                ma = max(target_target_v, source_target_v)
                if kind=='da_rel':
                    if mi  != ma:
                        dif = (source_target_da_v-mi) if met == 'acc' else (ma-source_target_da_v)
                        rel = dif/(ma  - mi)*100
                        # cap
                        res[i, j] = (rel if rel >= -100 else -100) if rel <= 100 else 100
                    else:
                        res[i, j] = 100 if source_target_da_v >= mi else -100
                #elif kind=='da_abs':
                #    res[i, j] = (source_target_da_v - target_target_v)*100
                elif kind=='target_minus_source':
                    res[i, j] = (source_target_v - target_target_v)*100
                else:
                    raise BaseException("unknown kind", kind)
            else:
                raise BaseException("error {} {} {} {}".format(source, target, len(target_target), len(source_target), len(source_target_alpha)))
    return res

In [ ]:
acc_st = metric_m(acc_res, 'acc', kind='target_minus_source')
acc_da = metric_m(acc_res, 'acc', kind='da_rel')
#acc_alpha0_1 = metric_m(acc_res1, 'acc', kind='alpha_rel')
#acc_alpha0_2 = metric_m(acc_res2, 'acc', kind='alpha_rel')

#dp_st = metric_m(dp_res8, 'dp', kind='st_abs')
#dp_alpha0 = metric_m(dp_res8, 'dp', kind='alpha_rel')
#dp_alpha0_1 = metric_m(dp_res1, 'dp', kind='alpha_rel')
#dp_alpha0_2 = metric_m(dp_res2, 'dp', kind='alpha_rel')

#eop_st = metric_m(eop_res8, 'eop', kind='st_abs')
#eop_alpha0 = metric_m(eop_res8, 'eop', kind='alpha_rel')
#eop_alpha0_1 = metric_m(eop_res1, 'eop', kind='alpha_rel')
#eop_alpha0_2 = metric_m(eop_res2, 'eop', kind='alpha_rel')

In [ ]:
# better, equal cases, avg
np.mean(acc_da>0), np.mean(acc_da==0), np.nanmean(acc_da)

In [ ]:
# red points = degrade in accuracy 
label = "$(ACC_{ntdk}-ACC_{tt})\cdot 100$"
_, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(acc_st,xticklabels=False, yticklabels = False, #row_cluster=cluster, col_cluster=cluster,
                    cbar_kws={'label': label, 'shrink':0.6}, center=0, vmin=-5, vmax=5,
                    linewidths= 0.25, cmap="RdYlGn")
plt.savefig('results/fig2acc.png', bbox_inches='tight', dpi=400)
plt.show()

In [ ]:
# red points = degrade dp difference
label = "$(DP_{tt}-DP_{ntdk})\cdot 100$" #if met=='dp' else "$EOP_{ntdk}-EOP_{tt}$")
_, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(-dp_st,xticklabels=False, yticklabels = False, #row_cluster=cluster, col_cluster=cluster,
                    cbar_kws={'label': label, 'shrink':0.6}, center=0, vmin=-5, vmax=5,
                    linewidths= 0.25, cmap="RdYlGn")
plt.savefig('results/fig2dp.png', bbox_inches='tight', dpi=400)
plt.show()

In [ ]:
# red points = degrade EOP difference
label = "$(EOP_{tt}-EOP_{ntdk})\cdot 100$"
_, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(-eop_st,xticklabels=False, yticklabels = False, #row_cluster=cluster, col_cluster=cluster,
                    cbar_kws={'label': label, 'shrink':0.6}, center=0, vmin=-5, vmax=5,
                    linewidths= 0.25, cmap="RdYlGn")
plt.savefig('results/fig2eop.png', bbox_inches='tight', dpi=400)
plt.show()

In [ ]:
# compute dimensions of analysis
def sim(k, d, agg=np.mean, att=None):
    atts = dists[k]
    if att is None:
        return agg([v[d] for v in atts.values()])
    return atts[att][d]

def calc_agg(results, df):
    res = []
    for k in dists:
        sr, tg = k
        if sr in states and tg in states and sr != tg:
            res.append(
                { 'source':sr, 
                  'target':tg,
                  'ssize':dists[(sr, tg)]['AGEP']['len_s_train'], # 'AGEP' or any other att
                  'tsize':dists[(sr, tg)]['AGEP']['len_t_train'],
                  'avg_d_att':sim(k, 'd_att'),
                  'avg_d_y_cond':sim(k, 'd_y_cond'),
                  'avg_d_y_cond_est':sim(k, 'd_y_cond_est'),
                  'avg_d_splits':sim(k, 'd_splits'),
                  'min_d_y_cond':sim(k, 'd_y_cond', np.min),
                  'min_d_y_cond_est':sim(k, 'd_y_cond_est', np.min),
                  'st_w_dist':cond(results, source=sr, target=tg, da=False)[0]['w_dist'],
                  'da_w_dist':cond(results, source=sr, target=tg, da=True)[0]['w_dist'],
                  'met':df[state2pos[sr], state2pos[tg]]
                }
            )
    return res

In [ ]:
%%time
acc_agg = calc_agg(results=acc_res, df=acc_da)
#acc_agg1 = calc_agg(results=acc_res1, df=acc_alpha0_1)
#acc_agg2 = calc_agg(results=acc_res2, df=acc_alpha0_2)
#dp_agg = calc_agg(results=dp_res, df=dp_alpha0)
#dp_agg1 = calc_agg(results=dp_res1, df=dp_alpha0_1)
#dp_agg2 = calc_agg(results=dp_res2, df=dp_alpha0_2)
#eop_agg = calc_agg(results=eop_res, df=eop_alpha0)
#eop_agg1 = calc_agg(results=eop_res1, df=eop_alpha0_1)
#eop_agg2 = calc_agg(results=eop_res2, df=eop_alpha0_2)

In [ ]:
def plot_rel(res, label):
    #avg_d_att = np.array([v['avg_d_att'] for v in res])
    #avg_d_splits = np.array([v['avg_d_splits'] for v in res])
    #ssize = np.array([v['ssize'] for v in res])
    #tsize = np.array([v['tsize'] for v in res])
    xs = np.array([v['st_w_dist'] for v in res])
    ys = np.array([v['da_w_dist'] for v in res])
    cs = np.array([v['met'] for v in res])
    ds = np.array([v['min_d_y_cond'] for v in res])
    _, ax = plt.subplots(figsize=(8, 5))
    plt.scatter(xs, ys, c=cs, linewidths = .5, marker ="o", cmap="RdYlGn", s=20, norm=TwoSlopeNorm(0, -100, 100))
    #ax.plot([0, .25], [0, .25])
    plt.colorbar(label=label, orientation="vertical", shrink=0.7)
    plt.xlabel('$W(T_{ntdk})$')
    plt.ylabel('$W(T_{ftdk})$')
    plt.savefig('results/fig3'+label.lower()+'.png', bbox_inches='tight', dpi=400)
    plt.show()
    return xs, ys, cs, ds

acc_xs, acc_ys, acc_cs, acc_ds = plot_rel(acc_agg, label="rACC")

In [ ]:
def plot_res_zoom(xs, ys, cs):
    pos = np.where( (xs < .05) & (ys < .05) )[0]
    xs1 = xs[pos]
    ys1 = ys[pos]
    cs1 = cs[pos]

    _, ax = plt.subplots(figsize=(7, 6))
    plt.scatter(xs1, ys1, c=cs1, linewidths = .5, marker ="o", cmap="RdYlGn", s=20, norm=TwoSlopeNorm(0, -100, 100))
    plt.colorbar()
    plt.show()
    return xs1, ys1, cs1
    
_, _, acc_cs1 = plot_res_zoom(acc_xs, acc_ys, acc_cs)

In [ ]:
def plot_res_dist(cs, cs1):
    data1 = cs
    data1 = data1[~np.isnan(data1)]
    print(np.mean(data1))
    print(stats.ttest_1samp(data1, popmean=0))
    sns.kdeplot(np.array(data1), bw_method=0.5)

    data2 = cs1
    data2 = data2[~np.isnan(data2)]
    print(np.mean(data2))
    print(stats.ttest_1samp(data2, popmean=0))
    sns.kdeplot(np.array(data2), bw_method=0.5)
    plt.show()
    
plot_res_dist(acc_cs, acc_cs1)

In [ ]:
dp_xs, dp_ys, dp_cs, dp_ds = plot_rel(dp_agg, label="rDP")

In [ ]:
eop_xs, eop_ys, eop_cs, eop_ds = plot_rel(eop_agg, label="rEOP")

In [ ]:
def plot_rel_p(xs, cs1, ys, cs2, ds, cs3, es=None, cs4=None, fname='unknown', xlabel='maximum distance', ylabel="Mean rACC", 
               label1='$W(T_{ntdk})$', label2='$W(T_{ftdk})$', label3='$W( \hat{P}^{X_w}(Y), P_T(Y) )$', 
               label4='unknown', thr=0.03):
    if xs is not None:
        sortby = np.round(xs, 3)
        pos = np.argsort(sortby)
        ns = range(1, 1+len(pos))
        posf = np.cumsum(cs1[pos])/ns
        plt.plot(sortby[pos], posf, label=label1)
        print(ylabel, 'at', label1, '<=', thr, posf[sum(sortby<=thr)])

    if ys is not None:
        sortby = np.round(ys, 3)
        pos = np.argsort(sortby)
        ns = range(1, 1+len(pos))
        posf = np.cumsum(cs2[pos])/ns
        plt.plot(sortby[pos], posf, label=label2)
        print(ylabel, 'at', label2, '<=', thr, posf[sum(sortby<=thr)])

    if ds is not None:
        sortby = np.round(ds, 3)
        pos = np.argsort(sortby)
        ns = range(1, 1+len(pos))
        posf = np.cumsum(cs3[pos])/ns
        plt.plot(sortby[pos], posf, label=label3)
        print(ylabel, 'at', label3, '<=', thr, posf[sum(sortby<=thr)])

    if es is not None:
        sortby = np.round(es, 3)
        pos = np.argsort(sortby)
        ns = range(1, 1+len(pos))
        posf = np.cumsum(cs4[pos])/ns
        plt.plot(sortby[pos], posf, label=label4)
        print(ylabel, 'at', label4, '<=', thr, posf[sum(sortby<=thr)])

    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.legend(loc='upper right')
    plt.ylim(0)
    plt.savefig('results/fig4'+fname+'.png', bbox_inches='tight', dpi=400)
    plt.show()

In [ ]:
#plot_rel_p(acc_xs, acc_cs, acc_ys, acc_cs, acc_ds, acc_cs, fname="racc") # this add the ntdk series
plot_rel_p(None, None, acc_ys, acc_cs, acc_ds, acc_cs, fname="racc")

In [ ]:
acc_xs1, acc_ys1, acc_cs1, acc_ds1 = plot_rel(acc_agg1, label="rACC")
acc_xs2, acc_ys2, acc_cs2, acc_ds2 = plot_rel(acc_agg2, label="rACC")
#acc_xs3, acc_ys3, acc_cs3, acc_ds3 = plot_rel(acc_agg3, label="rACC")

In [ ]:
plot_rel_p(acc_ds, acc_cs, acc_ds2, acc_cs2, acc_ds1, acc_cs1, xlabel = 'maximum distance $W( \hat{P}^{X_w}(Y), P_T(Y) )$',
           fname="racc3", ylabel="Mean rACC", label1='na=6', label2='na=3', label3='na=2')

In [ ]:
plot_rel_p(dp_xs, dp_cs, dp_ys, dp_cs, dp_ds, dp_cs, ylabel="Mean rDP", fname="rdp")

In [ ]:
#plot_rel_p(dp_xs, dp_cs, dp_ys, dp_cs, dp_ds, dp_cs, ylabel='rDP', fname="rdp")

In [ ]:
plot_rel_p(acc_ds, acc_cs, dp_ds, dp_cs, eop_ds, eop_cs, xlabel = 'maximum distance $W( \hat{P}^{X_w}(Y), P_T(Y) )$',
           fname='rrr', ylabel="rACC-rDP-rEOP", label1='rACC', label2='rDP', label3='rEOP')

In [ ]:
dp_xs1, dp_ys1, dp_cs1, dp_ds1 = plot_rel(dp_agg1, label="rDP")
dp_xs2, dp_ys2, dp_cs2, dp_ds2 = plot_rel(dp_agg2, label="rDP")
eop_xs1, eop_ys1, eop_cs1, eop_ds1 = plot_rel(eop_agg1, label="rEOP")
eop_xs2, eop_ys2, eop_cs2, eop_ds2 = plot_rel(eop_agg2, label="rEOP")

In [ ]:
plot_rel_p(dp_ds, dp_cs, dp_ds2, dp_cs2, dp_ds1, dp_cs1, xlabel = 'maximum distance $W( \hat{P}^{X_w}(Y), P_T(Y) )$', 
           fname="rdp3", ylabel="Mean rDP", label1='na=6', label2='na=3', label3='na=2')

In [ ]:
plot_rel_p(eop_ds, eop_cs, eop_ds2, eop_cs2, eop_ds1, eop_cs1, xlabel = 'maximum distance $W( \hat{P}^{X_w}(Y), P_T(Y) )$', 
           fname="reop3", ylabel="Mean rEOP", label1='na=6', label2='na=3', label3='na=2')